# Part 2: Time Series Modeling

In this notebook, you will implement functions to extract features from time series data and build ARIMA models.

In [1]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.arima.model import ARIMA
from pathlib import Path
import os
from pathlib import Path
import matplotlib.pyplot as plt

# Set style for plots
plt.style.use('seaborn')
%matplotlib inline

/var/folders/91/318xx77d4wbb95s3dsdtsshc0000gq/T/ipykernel_28229/1873047089.py:13: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


## 1. Feature Extraction

Implement the `extract_time_series_features` function to calculate rolling window features.

In [2]:
def extract_time_series_features(data, window_size=60):
    """
    Extract rolling window features from time series data.

    Parameters
    ----------
    data : pd.DataFrame
        Must include 'timestamp', 'subject_id', 'session',
        and physiological signals like 'heart_rate', 'eda', 'temperature'.
    window_size : int
        Size of the rolling window in seconds.

    Returns
    -------
    pd.DataFrame
        DataFrame containing extracted features for each signal:
        mean, std, min, max, and autocorrelation at lag 1.
    """

    def autocorr_lag1(x):
        if len(x) < 2 or x.isnull().all():
            return np.nan
        return x.autocorr(lag=1)

    data['timestamp'] = pd.to_datetime(data['timestamp'])
    data = data.sort_values(by=['subject_id', 'session', 'timestamp'])
    data = data.set_index('timestamp')

    signal_columns = ['heart_rate', 'eda', 'temperature']
    features = []

    grouped = data.groupby(['subject_id', 'session'])

    for (subject_id, session), group in grouped:
        rolled = group[signal_columns].rolling(f'{window_size}s', min_periods=1)
        feature_df = rolled.agg(['mean', 'std', 'min', 'max', autocorr_lag1])
        feature_df.columns = [
            f"{col[0]}_{col[1].lower() if isinstance(col[1], str) else 'autocorr'}"
            for col in feature_df.columns
        ]
        feature_df['subject_id'] = subject_id
        feature_df['session'] = session
        features.append(feature_df.reset_index())

    result = pd.concat(features, ignore_index=True)
    return result


In [3]:
import pandas as pd
from pathlib import Path

data_dir = Path('/Users/hteshome/Desktop/4-it-s-about-time-haile-teshome/processed_data/')
all_files = list(data_dir.glob("S*_processed.csv"))
dataframes = [pd.read_csv(file) for file in all_files]
preprocessed_data = pd.concat(dataframes, ignore_index=True)
preprocessed_data['timestamp'] = pd.to_datetime(preprocessed_data['timestamp'])
time_domain_df = extract_time_series_features(preprocessed_data, window_size=60)
time_domain_df.head()


/Users/hteshome/anaconda3/lib/python3.10/site-packages/numpy/lib/function_base.py:2845: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/Users/hteshome/anaconda3/lib/python3.10/site-packages/numpy/lib/function_base.py:2845: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/Users/hteshome/anaconda3/lib/python3.10/site-packages/numpy/lib/function_base.py:2845: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/Users/hteshome/anaconda3/lib/python3.10/site-packages/numpy/lib/function_base.py:2845: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/Users/hteshome/anaconda3/lib/python3.10/site-packages/numpy/lib/function_base.py:2845: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/Users/hteshome/anaconda3/lib/python3.10/site-packages/numpy/lib/function_base.py:2845: RuntimeWarning: Degrees of freedom <=

,timestamp,heart_rate_mean,heart_rate_std,heart_rate_min,heart_rate_max,heart_rate_autocorr_lag1,eda_mean,eda_std,eda_min,eda_max,eda_autocorr_lag1,temperature_mean,temperature_std,temperature_min,temperature_max,temperature_autocorr_lag1,subject_id,session
0,2018-12-05 16:29:07,116.000000,NaN,116.0,116.0,NaN,0.000000,NaN,0.0,0.000000,NaN,21.89,NaN,21.89,21.89,NaN,S1,Final
1,2018-12-05 16:29:08,99.250000,23.688077,82.5,116.0,NaN,0.002563,0.003624,0.0,0.005125,NaN,21.89,0.0,21.89,21.89,NaN,S1,Final
2,2018-12-05 16:29:09,98.276667,16.834626,82.5,116.0,-1.000000,0.008542,0.010669,0.0,0.020501,1.000000,21.89,0.0,21.89,21.89,NaN,S1,Final
3,2018-12-05 16:29:10,95.270000,15.003220,82.5,116.0,-0.936368,0.011852,0.010942,0.0,0.021783,0.741535,21.89,0.0,21.89,21.89,NaN,S1,Final
4,2018-12-05 16:29:11,95.936000,13.078235,82.5,116.0,-0.909283,0.014095,0.010720,0.0,0.023064,0.794006,21.89,0.0,21.89,21.89,NaN,S1,Final


## 2. ARIMA Modeling

Implement the `build_arima_model` function to fit ARIMA models and generate diagnostic plots.

In [4]:
def build_arima_model(series, order=(1, 1, 1), output_dir='plots',
                      subject_id='unknown', session='unknown', signal_name='signal'):
    """
    Fit an ARIMA model on a univariate time series and save diagnostic plots.

    Parameters
    ----------
    series : pd.Series
        The input univariate time series (indexed by timestamp).
    order : tuple
        ARIMA model order (p,d,q).
    output_dir : str
        Path to save plots.
    subject_id : str
        Subject ID to include in plot filenames.
    session : str
        Session name to include in plot filenames.
    signal_name : str
        Signal name to include in plot filenames.

    Returns
    -------
    model_fit : ARIMAResultsWrapper
        Fitted ARIMA model object (with predict and fit methods).
    """


    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)

    model = ARIMA(series, order=order)
    model_fit = model.fit()

    diag_fig = model_fit.plot_diagnostics(figsize=(10, 6))
    diag_filename = f"{subject_id}_{session}_{signal_name}_arima_diagnostics.png".replace(" ", "_")
    diag_fig.savefig(output_path / diag_filename)
    plt.close(diag_fig)

    residuals = model_fit.resid
    fig, ax = plt.subplots(figsize=(10, 4))
    ax.plot(residuals, label="Residuals")
    ax.axhline(0, color='gray', linestyle='--')
    ax.set_title(f"ARIMA Residuals - {subject_id} - {session} - {signal_name}")
    ax.set_xlabel("Time")
    ax.set_ylabel("Residual")
    ax.legend()
    res_filename = f"{subject_id}_{session}_{signal_name}_arima_residuals.png".replace(" ", "_")
    fig.savefig(output_path / res_filename)
    plt.close(fig)

    return model_fit



In [5]:
# What subject / session combos are present and how many non-NaN points in each?
(
    time_domain_df
      .groupby(['subject_id', 'session'])['heart_rate_mean']
      .apply(lambda x: x.notna().sum())     
      .reset_index(name='n_valid_points')
      .sort_values('n_valid_points', ascending=False)
      .head(10)
)

sid = "S001"         
sess = 1            

series = (
    time_domain_df
      .query("subject_id == @sid and session == @sess")
      .set_index('timestamp')['heart_rate_mean']
      .asfreq('60s')                
      .interpolate(limit_direction='both')
      .dropna()
)

# Show all subject/session pairs and valid point counts
available = (
    time_domain_df
    .groupby(['subject_id', 'session'])['heart_rate_mean']
    .apply(lambda x: x.notna().sum())
    .reset_index(name='n_valid_points')
    .sort_values('n_valid_points', ascending=False)
)

if len(series) < 10:
    print(f"Skipping ARIMA for {sid} - {sess} (not enough data)")
else:
    model = build_arima_model(series, order=(1, 1, 1), output_dir='plots')

print(available)

sid = available.iloc[0]['subject_id']
sess = available.iloc[0]['session']

series = (
    time_domain_df
    .query("subject_id == @sid and session == @sess")
    .set_index('timestamp')['heart_rate_mean']
    .asfreq('60s')  # force regular time steps
    .interpolate(limit_direction='both')
    .dropna()
)

print("After prep:", len(series), "points")


print("after prep:", len(series), "points")    # sanity check
if len(series) < 10:             # <-- rule of thumb: need ≥ p+q+d+1 obs
    raise ValueError("Not enough data; try a different subject/session "
                     "or aggregate several sessions first.")

model = build_arima_model(series, order=(1,1,1), output_dir='plots')


Skipping ARIMA for S001 - 1 (not enough data)
   subject_id    session  n_valid_points
9          S3      Final           25812
6          S2      Final           25327
18         S6      Final           23912
0          S1      Final           23387
3         S10      Final           23064
21         S7      Final           19639
24         S8      Final           17887
15         S5      Final           15240
27         S9      Final           14197
20         S6  Midterm 2           14181
8          S2  Midterm 2           13875
5         S10  Midterm 2           12982
28         S9  Midterm 1           12663
12         S4      Final           12560
29         S9  Midterm 2           12422
14         S4  Midterm 2           12408
22         S7  Midterm 1           12374
10         S3  Midterm 1           12203
17         S5  Midterm 2           12007
7          S2  Midterm 1           11975
16         S5  Midterm 1           11974
4         S10  Midterm 1           11678
1          

In [6]:
model = build_arima_model(
    series,
    order=(1, 1, 1),
    output_dir='plots',
    subject_id='S001',
    session='Midterm_1',
    signal_name='heart_rate'
)

print(model.summary())


                               SARIMAX Results                                
Dep. Variable:        heart_rate_mean   No. Observations:                  431
Model:                 ARIMA(1, 1, 1)   Log Likelihood               -1636.190
Date:                Wed, 11 Jun 2025   AIC                           3278.379
Time:                        11:16:47   BIC                           3290.571
Sample:                    12-05-2018   HQIC                          3283.193
                         - 12-05-2018                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.5432      0.036     15.180      0.000       0.473       0.613
ma.L1         -0.9317      0.023    -41.180      0.000      -0.976      -0.887
sigma2       117.9269      5.417     21.772      0.0